In [19]:
import numpy as np
import imageio
import cv2
import matplotlib.pyplot
import imutils
from PIL import Image

In [20]:
class NeuralNetwork:
    
    #initialise neural network
    def __init__(self, givenINodes, givenHNodes, givenONodes, givenLearningRate):
        #setup number of nodes in each layer
        self.inputNodes = givenINodes
        self.hiddenNodes = givenHNodes
        self.outputNodes = givenONodes
        
        #setup weights with initial values
        self.weightsInputHidden = np.random.normal(0.0, pow(self.inputNodes, -0.5), (self.hiddenNodes, self.inputNodes))
        self.weightsHiddenOutput = np.random.normal(0.0, pow(self.hiddenNodes, -0.5), (self.outputNodes, self.hiddenNodes))
        
        #setup learning rate
        self.learningRate = givenLearningRate
        
        #setup activation function
        self.activationFunc = lambda x: self.sigmoid(x)
            
        pass
            
    def sigmoid(self, Z):
        A = 1 / (1 + np.exp(-Z))
        return A
    
    #train the neural network
    def train(self, givenInputs, givenTargets):
        #inputs and target lists turned into 2D arrays
        inputs = np.array(givenInputs, ndmin=2).T
        targets = np.array(givenTargets, ndmin=2).T
        
        #signals into hidden layer calculated
        hiddenInputs = np.dot(self.weightsInputHidden, inputs)
        #signals coming from hidden layer calculated
        hiddenOutputs = self.activationFunc(hiddenInputs)
        
        #signals into output layer calculated
        finalInputs = np.dot(self.weightsHiddenOutput, hiddenOutputs)
        #signals from output layer calculated
        finalOutputs = self.activationFunc(finalInputs)
        
        #error backpropagation:
        #errors from output layer
        outputErrors = targets - finalOutputs
        #errors from hidden layer
        hiddenErrors = np.dot(self.weightsHiddenOutput.T, outputErrors)
        
        #weight links updated between all layers
        self.weightsHiddenOutput += self.learningRate * np.dot((outputErrors * finalOutputs * (1.0 - finalOutputs)), np.transpose(hiddenOutputs))
        self.weightsInputHidden += self.learningRate * np.dot((hiddenErrors * hiddenOutputs * (1.0 - hiddenOutputs)), np.transpose(inputs))
        
        pass
    
    #query the neural network
    def query(self, givenInputs):
        #inputs list converted into a 2D array
        inputs = np.array(givenInputs, ndmin=2).T
        
        #signals into hidden layer calculated
        hiddenInputs = np.dot(self.weightsInputHidden, inputs)
        #signals coming from hidden layer calculated
        hiddenOutputs = self.activationFunc(hiddenInputs)
        
        #signals into output layer calculated
        finalInputs = np.dot(self.weightsHiddenOutput, hiddenOutputs)
        #final signals from neural net calculated
        finalOutputs = self.activationFunc(finalInputs)
        
        return finalOutputs
    
        

In [21]:
def makeSquare(imageName):
    #input image
    im = Image.open('separated/'+imageName)
    #make the image a square shape
    sqrWidth = np.ceil(np.sqrt(im.size[0]*im.size[1])).astype(int)
    im_resize = im.resize((sqrWidth, sqrWidth))
    #save
    im_resize.save('output.png')
    pass

In [22]:
def identifyChar(imageName):

    #set number of nodes in each layer
    inputNodes = 784
    hiddenNodes = 200
    outputNodes = 10

    #set learning rate
    learningRate = 0.37

    #an instance of the neural network created
    n = NeuralNetwork(inputNodes, hiddenNodes, outputNodes, learningRate)
    
    #trainPrep

    #open training data file in READ mode
    trainingDataFile = open("mnist_dataset/mnist_train_100.csv",'r')
    #save all the lines from the file into a list
    trainingDataList = trainingDataFile.readlines()
    #close the file
    trainingDataFile.close()

    #number of times training data used for training
    epochs = 10

    for e in range(epochs):
        #for every record in the training data
        for record in trainingDataList:
            #split each record by the commas
            allVals = record.split(',')
            #scale and shift the inputs
            inputs = (np.asfarray(allVals[1:]) / 255.0 * 0.99) + 0.01

            #target output values created
            targets = np.zeros(outputNodes) + 0.01

            targets[int(allVals[0])] = 0.99
            #neural net trained using inputs and targets 
            n.train(inputs, targets)

            pass
        pass

    
    print ("Loading image...")
    makeSquare(imageName)
    
    #read in square image
    imageArray = imageio.imread('output.png', as_gray=True)

    #rotate image
    imageRotated = imutils.rotate(imageArray, 360)

    #make image 28 by 28 pixels
    imageResized = cv2.resize(imageRotated, (28, 28), cv2.INTER_AREA)

    #scale and shift image inputs
    imageData = 255.0 - imageResized.reshape(784)
    imageData = (imageData / 255.0 * 0.99) + 0.01

    print ("Min:", np.min(imageData))
    print ("Max:", np.max(imageData))

    #display image
    matplotlib.pyplot.imshow(imageData.reshape(28,28), cmap='Greys', interpolation='None')

    #query the neural net
    outputs = n.query(imageData)

    print (outputs)

    label = np.argmax(outputs)
    print ("Predicted value:", label)
    
    return label
